In [1]:
import pandas as pd
import requests, re, time
from bs4 import BeautifulSoup

In [44]:
url = "https://www.amazon.com/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A2811119011%2Cn%3A7072561011&dc&ds=v1%3ALg%2BI7W8x3qOCmrDsTOUseTdkjVGEHCRYQPlZKCjiQlo&qid=1662655090&rnid=2811119011&ref=sr_nr_n_1"
mainUrl = 'https://www.amazon.com'

In [3]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.62',
            'Accept-Language': 'en-US, en;q=0.5'})

In [4]:
# getting the url response

response = requests.get(url=url, headers=headers)

In [5]:
# getting the response content

primaryPageSoup = BeautifulSoup(markup=response.content, features='lxml')

In [6]:
# finding the container 

parentDiv = primaryPageSoup.find(name="div", 
                                attrs={'class': 's-main-slot s-result-list s-search-results sg-row'})

In [7]:
# finding the card with phone titles

phoneDivs = parentDiv.findChildren(name="div", attrs={'data-uuid': True},
                                   recursive=False)

Finding children <b>recursively</b> entails getting *their* grandchildren too, as _viable_ results.

In [8]:
# Getting title information

titleCard = phoneDivs[0].find(name='h2').text
print(titleCard)

TCL 30 SE |2022| Unlocked Cell Phone 128GB ROM + 4GB RAM, 6.52" Android Phone GSM Unlocked Smartphone with 50MP Camera, 5000mAh, US Version, Space Gray(Not Compatible with Verizon/Boost/Sprint)  


In [9]:
redirectUrl = phoneDivs[0].find(name='a')['href']

In [11]:
pageRedirect = requests.get(mainUrl + redirectUrl, headers=headers)

In [12]:
redirectSoup = BeautifulSoup(markup=pageRedirect.content, features='lxml')

In [13]:
infoTable = redirectSoup.find(name='table', 
                              attrs={'class': 'a-normal a-spacing-micro'})

In [14]:
brandRow = infoTable.find(name='tr', 
                          attrs={'class': 'a-spacing-small po-brand'})

In [15]:
brand = brandRow.findChildren(name='td', recursive=False)[1].text.strip()

In [16]:
modelRow = infoTable.find(name='tr', 
                          attrs={'class': 'a-spacing-small po-model_name'})

In [17]:
model = modelRow.findChildren(name='td', recursive=False)[1].text.strip()

In [18]:
opSystemrow = infoTable.find(name='tr', 
                             attrs={'class': 'a-spacing-small po-operating_system'})

In [19]:
operatingSystem = opSystemrow.findChildren(name='td', recursive=False)[1].text.strip()

In [20]:
memoryStorageRow = infoTable.find(name='tr', 
                                  attrs={'class': 'a-spacing-small po-memory_storage_capacity'})

In [21]:
memoryStorage = memoryStorageRow.findChildren(name='td', recursive=False)[1].text.strip()

In [22]:
displaySizeRow = infoTable.find(name='tr',
                                attrs={'class': 'a-spacing-small po-display.size'})

In [23]:
displaySize = displaySizeRow.findChildren(name="td", recursive=False)[1].text.strip()

In [24]:
connectorTypeRow = infoTable.find(name="tr", attrs={'class': 'a-spacing-small po-connector_type'})

In [25]:
connectorType = connectorTypeRow.findChildren(name="td", recursive=False)[1].text.strip()

AttributeError: 'NoneType' object has no attribute 'findChildren'

In [26]:
print(model, brand, memoryStorage)

TCL 30 SE TCL 128 GB


We can generalize this process for each card and collect:
- the brand name
- the model of the device
- the memory storage
- the screen/display size, and
- the connector type

In [43]:
paginationSpan = primaryPageSoup.find(name='span', attrs={'class': 's-pagination-strip'})


pageNum = 2

paginationSpan.find(name='a', attrs={'aria-label': f'Go to next page, page {str(pageNum)}'})['href']

'/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A2811119011%2Cn%3A7072561011&dc&page=2&qid=1662992493&rnid=2811119011&ref=sr_pg_1'

Ratings by <b>*features*</b>.
> to be implemented

In [27]:
# Grabbing link to reviews
reviewsUrl = redirectSoup.find(name="a", 
                               attrs={'data-hook': 'see-all-reviews-link-foot'})['href']

## Generalizaing Pagination for Reviews

In [28]:
# Pagination
pageCounter = 0
collectedReviews = []

while pageCounter < 5:
    nextUrl = reviewsUrl
    # Getting request for the review page
    reviewsPage = requests.get(url=mainUrl + reviewsUrl,
                          headers=headers)
    # Creating the Soup
    reviewsSoup = BeautifulSoup(reviewsPage.content, features='lxml')
    
    # Obtaining the reviews parent div
    reviewsDiv = reviewsSoup.find(name="div", 
                               attrs={'id': 'cm_cr-review_list'})
    
    # All reviews in their respective divs
    reviewsList = reviewsDiv.findChildren(name='div',
                                     attrs={'data-hook': 'review'},
                                     recursive=False)
    for reviewDivs in reviewsList:
        text = reviewsDiv.find(name="span", attrs={'data-hook': 'review-body'}).text.strip('\n')
        collectedReviews.append(text)
    try:   
        # Getting next page url
        paginationContainer = reviewsSoup.find(name="ul", attrs={'class': 'a-pagination'})
        nextUrl = paginationContainer.findChildren(name="li", recursive=False)[1].find(name="a")['href']
    except AttributeError:
        break
    
    # Sleeping to avoid making too many requests
    time.sleep(2)
    
    # Updating the page counter
    pageCounter += 1

In [29]:
collectedReviews.__len__()

50

In [ ]:
brandList = []
modelList = []
memoryList = []
operatingList = []
displayList = []

for i in range(10):
    response = requests.get(url=url, headers=headers)
    
    # getting the response content

    primaryPageSoup = BeautifulSoup(markup=response.content, features='lxml')
    
    # finding the container 

    parentDiv = primaryPageSoup.find(name="div", 
                                     attrs={'class': 's-main-slot s-result-list s-search-results sg-row'})
    
    # finding the card with phone titles

    phoneDivs = parentDiv.findChildren(name="div", attrs={'data-uuid': True}, recursive=False)
    
    for phone in phoneDivs:
        
        redirectUrl = phone.find(name='a')['href']
        
        pageRedirect = requests.get(mainUrl + redirectUrl, headers=headers)
        
        redirectSoup = BeautifulSoup(markup=pageRedirect.content, features='lxml')
        
        infoTable = redirectSoup.find(name='table', attrs={'class': 'a-normal a-spacing-micro'})
        
        brandRow = infoTable.find(name='tr', attrs={'class': 'a-spacing-small po-brand'})

        brand = brandRow.findChildren(name='td', recursive=False)[1].text.strip()
        brandList.append(brand)

        modelRow = infoTable.find(name='tr', attrs={'class': 'a-spacing-small po-model_name'})

        model = modelRow.findChildren(name='td', recursive=False)[1].text.strip()
        modelList.append(model)

        opSystemrow = infoTable.find(name='tr', 
                                     attrs={'class': 'a-spacing-small po-operating_system'})

        operatingSystem = opSystemrow.findChildren(name='td', recursive=False)[1].text.strip()
        operatingList.append(operatingSystem)

        memoryStorageRow = infoTable.find(name='tr', 
                                          attrs={'class': 'a-spacing-small po-memory_storage_capacity'})

        memoryStorage = memoryStorageRow.findChildren(name='td', recursive=False)[1].text.strip()
        memoryList.append(memoryStorage)

        displaySizeRow = infoTable.find(name='tr',
                                        attrs={'class': 'a-spacing-small po-display.size'})

        displaySize = displaySizeRow.findChildren(name="td", recursive=False)[1].text.strip()
        displayList.append(displaySize)
        
        # Grabbing link to reviews
        
        reviewsUrl = redirectSoup.find(name="a", 
                                       attrs={'data-hook': 'see-all-reviews-link-foot'})['href']
        
        # Pagination
        
        pageCounter = 0
        collectedReviews = []

        while pageCounter < 5:
            nextUrl = reviewsUrl
            # Getting request for the review page
            reviewsPage = requests.get(url=mainUrl + reviewsUrl,
                                  headers=headers)
            # Creating the Soup
            reviewsSoup = BeautifulSoup(reviewsPage.content, features='lxml')

            # Obtaining the reviews parent div
            reviewsDiv = reviewsSoup.find(name="div", 
                                       attrs={'id': 'cm_cr-review_list'})

            # All reviews in their respective divs
            reviewsList = reviewsDiv.findChildren(name='div',
                                             attrs={'data-hook': 'review'},
                                             recursive=False)
            for reviewDivs in reviewsList:
                text = reviewsDiv.find(name="span", attrs={'data-hook': 'review-body'}).text.strip('\n')
                collectedReviews.append(text)
            try:   
                # Getting next page url
                paginationContainer = reviewsSoup.find(name="ul", attrs={'class': 'a-pagination'})
                nextUrl = paginationContainer.findChildren(name="li", recursive=False)[1].find(name="a")['href']
            except AttributeError:
                break

            # Sleeping to avoid making too many requests
            time.sleep(2)

            # Updating the page counter
            pageCounter += 1
        
        
    paginationSpan = primaryPageSoup.find(name='span', attrs={'class': 's-pagination-strip'})


    pageNum = i

    paginationSpan.find(name='a', attrs={'aria-label': f'Go to next page, page {str(pageNum)}'})['href']